[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/tabular-classification/xgboost/xgboost.ipynb)


# <a id="top">Tabular classification using XGBoost</a>

This notebook illustrates how XGBoost models can be uploaded to the Openlayer platform.

**Important considerations:**
- **Categorical features.** From `xgboost>=1.5`, XGBoost introduced experimental support for [categorical data available for public testing](https://xgboost.readthedocs.io/en/latest/tutorials/categorical.html). We recommend encoding categorical features as illustrated in this notebook and **not** using the experimental feature with `enable_categorical=True` to upload models to Openlayer. The XGBoost package presented flaky behavior when such a feature is enabled and this is why it is discouraged for now. If this is critical to you, feel free to [reach out](mailto:support@openlayer.com)!
- **Feature dtypes.** XGBoost models are very sensitive to input data types. Some of the explainability techniques used by Openlayer rely on synthetic data generated by perturbing the original data samples. In that process, `int` values might be cast to `float` and if your XGBoost model was expecting an `int`, it will throw an error. To make sure that your model works well in the platform, make sure to **perform the casting inside the `predict_proba` function**, before creating the `xgb.DMatrix` and doing predictions with the model.

## <a id="toc">Table of contents</a>

1. [**Getting the data and training the model**](#1)
    - [Downloading the dataset](#download)
    - [Preparing the data](#prepare)
    - [Training the model](#train)
    

2. [**Using Openlayer's Python API**](#2)
    - [Instantiating the client](#client)
    - [Creating a project](#project)
    - [Uploading datasets](#dataset)
    - [Uploading models](#model)
        - [Shell models](#shell)
        - [Full models](#full-model)
    - [Committing and pushing to the platform](#commit)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/tabular-classification/xgboost/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Getting the data and training the model </a>

[Back to top](#top)

In this first part, we will get the dataset, pre-process it, split it into training and validation sets, and train a model. Feel free to skim through this section if you are already comfortable with how these steps look for an XGBoost model.   

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split

### <a id="download">Downloading the dataset </a>

We have stored the dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. Alternatively, you can also find the dataset on [this Kaggle competition](https://www.kaggle.com/datasets/uciml/mushroom-classification).

In [ ]:
%%bash

if [ ! -e "mushrooms.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/mushrooms.csv" --output "mushrooms.csv"
fi

In [ ]:
df = pd.read_csv("./mushrooms.csv")
df.head()

### <a id="prepare">Preparing the data</a>

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        print(f"encoding {feature}")
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
# replacing class names with 0 and 1
class_map = {"e": 0, "p": 1}

X, y = df.loc[:, df.columns != "class"], df[["class"]].replace(class_map)

In [ ]:
encoders = create_encoder_dict(X, list(X.columns))

X_enc_one_hot = data_encode_one_hot(X, encoders)
X_enc_one_hot

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

### <a id="train">Training the model</a>

In [ ]:
# Using XGBoost data format
dtrain = xgb.DMatrix(x_train_one_hot, label=y_train)
dval = xgb.DMatrix(x_val_one_hot, label=y_val)

In [ ]:
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2

xgboost_model = xgb.train(param, dtrain, num_round)

In [ ]:
preds = xgboost_model.predict(dval)
labels = dval.get_label()

In [ ]:
print(
    "error rate=%f"
    % (
        sum(1 for i in range(len(preds)) if int(preds[i] > 0.5) != labels[i])
        / float(len(preds))
    )
)

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="XGBoost project", 
    task_type=TaskType.TabularClassification,
    description="Evaluation of ML approaches"
)

### <a id="dataset">Uploading datasets</a>

Before adding the datasets to a project, we need to do two things:
1. Enhance the dataset with additional columns to make it comprehensive, such as adding a column for labels and one for model predictions (if you're uploading a model as well).
2. Prepare a `dataset_config.yaml` file. This is a file that contains all the information needed by the Openlayer platform to utilize the dataset. It should include the column names, the class names, etc. For details on the fields of the `dataset_config.yaml` file, see the [API reference](https://reference.openlayer.com/reference/api/openlayer.OpenlayerClient.add_dataset.html#openlayer.OpenlayerClient.add_dataset).

Let's start by enhancing the datasets with the extra columns:

In [ ]:
# Adding the column with the labels
training_set = x_train.copy(deep=True)
training_set["class"] = y_train.values
validation_set = x_val.copy(deep=True)
validation_set["class"] = y_val.values

In [ ]:
predict_proba = lambda x : [[1-p, p] for p in xgboost_model.predict(xgb.DMatrix(x))] 

In [ ]:
# Adding the column with the predictions (since we'll also upload a model later)
training_set["predictions"] = predict_proba(x_train_one_hot)
validation_set["predictions"] = predict_proba(x_val_one_hot)

Now, we can prepare the `dataset_config.yaml` files for the training and validation sets.

In [ ]:
# Some variables that will go into the `dataset_config.yaml` file
categorical_feature_names = feature_names # all features are categorical in this dataset
class_names = ["e", "p"]  # the classes on the dataset
column_names = list(training_set.columns)
feature_names = list(X.columns)  # feature names in the un-processed dataset
label_column_name = "class"
predictions_column_name = "predictions"

In [ ]:
import yaml 

# Note the camelCase for the dict's keys
training_dataset_config = {
    "categoricalFeatureNames": categorical_feature_names,
    "classNames": class_names,
    "columnNames": column_names,
    "featureNames":feature_names,
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionsColumnName": predictions_column_name,
}

with open("training_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(training_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

with open("validation_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(validation_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
# Training set
project.add_dataframe(
    dataset_df=training_set,
    dataset_config_file_path="training_dataset_config.yaml",
)

In [ ]:
# Validation set
project.add_dataframe(
    dataset_df=validation_set,
    dataset_config_file_path="validation_dataset_config.yaml",
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

When it comes to uploading models to the Openlayer platform, there are two options:

- The first one is to upload a **shell model**. Shell models are the most straightforward way to get started. They are comprised of metadata and all of the analysis are done via its predictions (which are [uploaded with the datasets](#dataset)).
- The second one is to upload a **full model**, with artifacts. When a full model is uploaded, it becomes available in the platform and it becomes possible to perform what-if analysis, use all the explainability techniques available, and perform a series of robustness assessments with it. 

#### <a id="shell">Shell models</a>

To upload a shell model, we only need to define its name, the architecture type, and add some metadata that will be rendered in the platform to help us identify it. This information should be saved to a `model_config.yaml` file.

Let's create a `model_config.yaml` file for our model:

In [ ]:
import yaml

model_config = {
    "name": "Mushroom classification model",
    "architectureType": "xgboost",
    "metadata": {  # Can add anything here, as long as it is a dict
        "objective_function": "Logistic",
        "max_depth": 2,
    }
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

In [ ]:
project.add_model(
    model_config_file_path="model_config.yaml",
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

Since in this example, we're interested in uploading a full model, let's unstage the shell model:

In [ ]:
project.restore("model")

#### <a id="full-model"> Full models </a>

To upload a model to Openlayer, you will need to create a model package, which is nothing more than a folder with all the necessary information to run inference with the model. The package should include the following:
1. A `requirements.txt` file listing the dependencies for the model.
2. Serialized model files, such as model weights, encoders, etc., in a format specific to the framework used for training (e.g. `.json` for XGBoost, `.pkl` for sklearn, `.pb` for TensorFlow, and so on.)
3. A `prediction_interface.py` file that acts as a wrapper for the model and implements the `predict_proba` function. 

Other than the model package, a `model_config.yaml` file is needed, with information about the model to the Openlayer platform, such as the framework used, feature names, and categorical feature names.


Lets prepare the model package one piece at a time
 

In [ ]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

**1. Adding the `requirements.txt` to the model package**

In [ ]:
!scp requirements.txt model_package

**2. Serializing the model and other objects needed**

In [ ]:
import pickle 

# Trained model
xgboost_model.save_model('model_package/model.json')

# Encoder for the categorical features
with open('model_package/encoders.pkl', 'wb') as handle:
    pickle.dump(encoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

**3. Writing the `prediction_interface.py` file**

In [ ]:
%%writefile model_package/prediction_interface.py

import pickle
from pathlib import Path

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

PACKAGE_PATH = Path(__file__).parent


class XgboostModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""
        self.model = xgb.Booster()
        self.model.load_model(PACKAGE_PATH / "model.json")
        
        with open(PACKAGE_PATH / "encoders.pkl", "rb") as encoders_file:
            self.encoders = pickle.load(encoders_file)

    def _data_encode_one_hot(self, df: pd.DataFrame) -> pd.DataFrame:
        """Pre-processing needed for our particular use case."""

        df = df.copy(True)
        df.reset_index(drop=True, inplace=True)  # Causes NaNs otherwise
        for feature, enc in self.encoders.items():
            enc_df = pd.DataFrame(
                enc.transform(df[[feature]]).toarray(),
                columns=enc.get_feature_names([feature]),
            )
            df = df.join(enc_df)
            df = df.drop(columns=feature)
        return df

    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""

        encoded_df = self._data_encode_one_hot(input_data_df)
        
        # Converting the data to the XGBoost data format
        data_xgb = xgb.DMatrix(encoded_df)
    
        # Making the predictions with the model
        preds = self.model.predict(data_xgb)
    
        # Post-processing the predictions to the format Openlayer expects
        preds_proba = [[1 - p, p] for p in preds]
        
        return preds_proba


def load_model():
    """Function that returns the wrapped model object."""
    return XgboostModel()

**Creating the `model_config.yaml`**

In [ ]:
import yaml 

model_config = {
    "name": "My XGBoost model",
    "architectureType": "xgboost",
    "classNames": class_names,
    "categoricalFeatureNames": categorical_feature_names,
    "featureNames":feature_names
}

with open('model_config.yaml', 'w') as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

Lets check that the model package contains everything needed:

In [ ]:
from openlayer.validators import ModelValidator

model_validator = ModelValidator(
    model_package_dir="model_package", 
    model_config_file_path="model_config.yaml",
    sample_data = validation_set[feature_names].iloc[:10, :]
)
model_validator.validate()

Now, we are ready to add the model:

In [ ]:
project.add_model(
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data = validation_set[feature_names].iloc[:10, :]
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Initial commit!")

In [ ]:
project.status()

In [ ]:
project.push()